<a href="https://colab.research.google.com/github/ami-iit/adam/blob/main/examples/mpc-ik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Mujoco, adam, and mediapy.

Download also mujoco-menagerie for the panda model and the urdf needed from adam.
Set some stuff for the visualization.

In [ ]:
!pip install mujoco
!pip install adam-robotics
!pip install -q mediapy
!git clone https://github.com/google-deepmind/mujoco_menagerie.git
!wget https://raw.githubusercontent.com/bulletphysics/bullet3/master/examples/pybullet/gym/pybullet_data/franka_panda/panda.urdf
# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl
%env OMP_NUM_THREADS=1

## Import packages

In [ ]:
import mujoco
import mediapy as media
from adam.casadi import KinDynComputations
import numpy as np
import casadi as cs

## Import the panda scene in mujoco

In [ ]:
# load scene from xml
model = mujoco.MjModel.from_xml_path("mujoco_menagerie/franka_emika_panda/scene.xml")


## Import urdf in adam

Set the commanded joint list and impor the urdf in adam.

For now I have to use a separate urdf for adam.
An importer for a mujoco model could be an idea for the future!

In [ ]:
joints_name_list = ['panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_joint8']

kindyn = KinDynComputations(urdfstring="panda.urdf", joints_name_list=joints_name_list)


## A wrapper interface with mujoco

In [ ]:
class MujocoWrapper:
  # a simple wrapper to use mujoco as a simulator
  def __init__(self, model, joints_list=None):
    self.model = model
    self.data = mujoco.MjData(model)
    self.renderer = mujoco.Renderer(self.model)

  def set_qpos(self, qpos):
    # set the joint positions
    self.data.qpos[:] = qpos
    mujoco.mj_forward(self.model, self.data)

  def get_qpos(self):
    # get the joint positions
    return self.data.qpos[:]

  def render(self):
    # render the scene and return the frame
    mujoco.mj_forward(self.model, self.data)
    self.renderer.update_scene(self.data)
    return self.renderer.render()

  def step(self):
    # step the simulation
    mujoco.mj_step(self.model, self.data)

  def set_qvel(self, qvel):
    # set the joint velocities
    self.data.qvel[:] = qvel
    mujoco.mj_forward(self.model, self.data)

wrapper = MujocoWrapper(model)


# Model Inverse Kinematics as an MPC

An MPC is maybe not the best way to solve an IK problem.
I just want to show how to use the casadi interface.

In [ ]:
# Create a casadi function for the forward kinematics of the end-effector
w_H_ee = kindyn.forward_kinematics_fun("panda_hand")

# casadi opti stack
opti = cs.Opti()

N = 10 # number of control intervals
dt = 0.05 # time step

# joints variables
q = opti.variable(8, N+1)
q_dot = opti.variable(8, N)
w_H_b = np.eye(4) # base of the manipulator as identity matrix

# set the desidered end-effector position as a parameter
# it will be set later when the mpc is solved at each iteration
des_ee_pos = opti.parameter(3)

# initial joint positions. It will be set at each iteration
q0 = opti.parameter(8)
opti.subject_to(q[:, 0] == q0)

target_cost = 0

for i in range(N):
    # integration - forward euler
    opti.subject_to(q[:, i+1] == q[:, i] + q_dot[:, i] * dt)
    # bounds on the joint velocities
    opti.subject_to(opti.bounded(-5, q_dot[:, i], 5))
    # running cost
    target_cost += cs.sumsqr(w_H_ee(w_H_b, q[:, i])[:3, 3] - des_ee_pos) * dt

# final cost
target_cost += cs.sumsqr(w_H_ee(w_H_b, q[:, -1])[:3, 3] - des_ee_pos)
# regularization on joint velocities as cost
velocities_cost = cs.sumsqr(q_dot) * 1e-3

# minimize the total cost
opti.minimize(target_cost + velocities_cost)

# set the solver
p_opts = {"expand": True, 'ipopt.print_level': 0, 'print_time': 0, 'ipopt.sb': 'yes'}
s_opts = {"max_iter": 100, "print_level": 0}
opti.solver("ipopt", p_opts, s_opts)


# Simulation loop

We set the joint velocities as control input in Mujoco.
We retrieve the joint positions as measurement from Mujoco and set them as starting state for the MPC.

On the notebook it is a bit slow.
To run it real time set OMP_NUM_THREADS=1 on your laptop!

In [ ]:
print("Simulation in progress. Wait a bit!")

# start mujoco simulation along with control
duration = 10  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.

# initial desired end-effector position
des_ee_pos_numeric = np.array([0.4, 1.0, 0.4])

frames = []
mujoco.mj_resetData(wrapper.model, wrapper.data)
i = 0
while wrapper.data.time < duration:
  wrapper.step()
  if len(frames) < wrapper.data.time * framerate:
    # you do not need to set the desired ee position every time step
    # you can set it only when you want to change it
    opti.set_value(des_ee_pos, des_ee_pos_numeric)
    q0_numeric = wrapper.get_qpos()
    # remove the last joint since they are not controlled
    q0_numeric = q0_numeric[:-1]
    # set the initial condition
    opti.set_value(q0, q0_numeric)
    sol = opti.solve()
    # take the q_dot solution at the first time step and extend with the last joint to 0
    sol_q_dot = sol.value(q_dot)[:, 0]
    sol_q_dot = np.concatenate((sol_q_dot, np.zeros(1)))
    wrapper.set_qvel(sol_q_dot)
    pixels = wrapper.render()
    frames.append(pixels)
    # set the solution as the initial condition for the next time step
    opti.set_initial(q, sol.value(q))
    opti.set_initial(q_dot, sol.value(q_dot))
    i += 1
    if wrapper.data.time > 2:
      # change the desired ee position
      des_ee_pos_numeric = np.array([2.0, 0.0, 0.2])
      opti.set_value(des_ee_pos, des_ee_pos_numeric)
    if wrapper.data.time > 4:
      # change the desired ee position
      des_ee_pos_numeric = np.array([0.0, 0.5, 0.4])
      opti.set_value(des_ee_pos, des_ee_pos_numeric)
    if wrapper.data.time > 6:
      # change the desired ee position
      des_ee_pos_numeric = np.array([0.0, -0.6, 0.6])
      opti.set_value(des_ee_pos, des_ee_pos_numeric)

media.show_video(frames, fps=framerate)
